In [1]:
import cv2
import mediapipe as mp

In [2]:
mphands=mp.solutions.hands
mpdraw=mp.solutions.drawing_utils
hands=mphands.Hands(max_num_hands=1)


In [ ]:
video = cv2.VideoCapture(0)

while True:
    suc, img = video.read()
    img1 = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    result = hands.process(img1)

    tipid = [4, 8, 12, 16, 20]

    if result.multi_hand_landmarks:
        for x in result.multi_hand_landmarks:

            lmlst = []
            for id, lm in enumerate(x.landmark):
                cx = lm.x
                cy = lm.y
                lmlst.append([id, cx, cy])

            if len(lmlst) == 21:
                fingercount = []

                gesture=""
                # Thumb
                if lmlst[12][1]<lmlst[20][1]: #left hand
                        if lmlst[4][1]>lmlst[3][1]:
                            fingercount.append(0)
                        else:
                            fingercount.append(1)
                else:   #right hand
                        if lmlst[4][1]<lmlst[3][1]:
                            fingercount.append(0)
                        else:
                            fingercount.append(1)

               
                for i in range(1, 5):
                    if lmlst[tipid[i]][2] < lmlst[tipid[i]-2][2]:
                        fingercount.append(1)
                    else:
                        fingercount.append(0)

               
                if fingercount == [1, 0, 0, 0, 0]:
                      
                     gesture= "THUMBS UP "
    
                elif  fingercount==[0,1,1,0,0]:
                   gesture="Victory"
                elif fingercount==[0,0,0,0,0]:
                    gesture="FIST"
                elif fingercount==[1,1,1,1,1]:
                    gesture="OPEN HAND"
                elif fingercount==[0,0,0,0,0]:
                    gesture="close hand"
                elif fingercount==[1,1,0,0,1]:
                    gesture="love you"
                elif fingercount==[0,1,0,0,0]:
                    gesture="Pointing up"
                elif fingercount==[1,0,0,0,1]:
                    gesture="rock n roll"
                elif fingercount==[0,0,1,1,1]:
                    gesture="Super"    
                elif fingercount==[0,1,1,1,1]:
                    gesture="Four"    

                cv2.putText(img,gesture,(35,400),cv2.FONT_HERSHEY_COMPLEX,2,(255,0,0),10) 

                import pyttsx3

                txt_sp=pyttsx3.init()
               
                voices = txt_sp.getProperty('voices')
                #female voice
                txt_sp.setProperty('voice', voices[1].id)
                txt_sp.setProperty('volume',0.50)
                txt_sp.say(gesture)

                txt_sp.runAndWait() 







            mpdraw.draw_landmarks(img, x, mphands.HAND_CONNECTIONS)

    cv2.imshow("image", img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

video.release()
cv2.destroyAllWindows()


In [5]:
import cv2
import mediapipe as mp
import pyttsx3

video = cv2.VideoCapture(0)

mpHands = mp.solutions.hands
hands = mpHands.Hands()
mpdraw = mp.solutions.drawing_utils

# Initialize TTS ONCE
engine = pyttsx3.init()
voices = engine.getProperty('voices')
engine.setProperty('voice', voices[1].id)
engine.setProperty('volume', 0.8)

prev_gesture = ""   # store last spoken gesture

tipid = [4, 8, 12, 16, 20]

while True:
    suc, img = video.read()
    if not suc:
        break

    img1 = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    result = hands.process(img1)

    gesture = ""

    if result.multi_hand_landmarks:
        for x in result.multi_hand_landmarks:
            lmlst = []
            for id, lm in enumerate(x.landmark):
                lmlst.append([id, lm.x, lm.y])

            if len(lmlst) == 21:
                fingercount = []

                # Thumb (left/right hand)
                if lmlst[12][1] < lmlst[20][1]:  # left hand
                    fingercount.append(1 if lmlst[4][1] < lmlst[3][1] else 0)
                else:  # right hand
                    fingercount.append(1 if lmlst[4][1] > lmlst[3][1] else 0)

                # Other fingers
                for i in range(1, 5):
                    fingercount.append(
                        1 if lmlst[tipid[i]][2] < lmlst[tipid[i]-2][2] else 0
                    )

                # Gesture mapping
                if fingercount == [1,0,0,0,0]:
                    gesture = "Thumbs Up"
                elif fingercount == [0,1,1,0,0]:
                    gesture = "Victory"
                elif fingercount == [0,0,0,0,0]:
                    gesture = "Fist"
                elif fingercount == [1,1,1,1,1]:
                    gesture = "Open Hand"
                elif fingercount == [1,1,0,0,1]:
                    gesture = "Love You"
                elif fingercount == [0,1,0,0,0]:
                    gesture = "Pointing Up"
                elif fingercount == [1,0,0,0,1]:
                    gesture = "Rock And Roll"
                elif fingercount == [0,0,1,1,1]:
                    gesture = "Super"
                elif fingercount == [0,1,1,1,1]:
                    gesture = "Four"

                # Speak ONLY if gesture changed
                if gesture != "" and gesture != prev_gesture:
                    engine.say(gesture)
                    engine.runAndWait()
                    prev_gesture = gesture

            mpdraw.draw_landmarks(img, x, mpHands.HAND_CONNECTIONS)

    cv2.putText(img, gesture, (30, 60),
                cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255, 0, 0), 3)

    cv2.imshow("Gesture Recognition", img)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

video.release()
cv2.destroyAllWindows()
